In [29]:
# accounts
import json
import solana
import time
import sys
import glob
from solana.rpc.api import Client
from solana.blockhash import Blockhash
from solana.publickey import PublicKey
from solana.system_program import CreateAccountWithSeedParams, create_account_with_seed, CreateAccountParams, create_account
from solana.transaction import Transaction, AccountMeta, TransactionInstruction
from solana.system_program import SYS_PROGRAM_ID
from spl.token.client import Token
from solana import keypair
from solana.publickey import PublicKey
from solana.sysvar import SYSVAR_RENT_PUBKEY

In [30]:
TOKEN_PROGRAM_ID: PublicKey = PublicKey("TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA")

first_key_in_key_folder = glob.glob("../keys/*")[0]
payer_loaded_account = solana.keypair.Keypair(json.load(open(first_key_in_key_folder))[:32])


http_client = Client("https://api.devnet.solana.com")
payer_public_key = payer_loaded_account.public_key



# todo: change program_id to system program, change data size =0, change lampord for both alice and bob
tx = Transaction()
alice_keypair = solana.keypair.Keypair.generate()
alice_pubkey = alice_keypair.public_key



tx = Transaction()
bob_keypair = solana.keypair.Keypair.generate()
bob_pubkey = bob_keypair.public_key


print(f"Payer Account: {payer_public_key}")
print(f"alice Account: {alice_pubkey}")
print(f"bob Account: {bob_pubkey}")

Payer Account: 763cSApzBT5scjhSd6BdnkKkMYBhiQmPn2gctswkdkJM
alice Account: 7pkWpUjuxwQ73KRoz2aqF3vwTGaFMhN8jwwVkTT7GcX1
bob Account: EJEumLvrmuGeCtfe9vMBPKG9oQVBMsA3J4ZELGh6zXe9


In [31]:
X_mint_account_address = Token.create_mint(conn= http_client, payer = payer_loaded_account, mint_authority = payer_loaded_account.public_key, decimals = 0, program_id = TOKEN_PROGRAM_ID, )
Y_mint_account_address = Token.create_mint(conn= http_client, payer = payer_loaded_account, mint_authority = payer_loaded_account.public_key, decimals = 0, program_id = TOKEN_PROGRAM_ID, )
alice_X_token_account = X_mint_account_address.create_associated_token_account(alice_pubkey)
alice_Y_token_account = Y_mint_account_address.create_associated_token_account(alice_pubkey)
bob_X_token_account = X_mint_account_address.create_associated_token_account(bob_pubkey)
bob_Y_token_account = Y_mint_account_address.create_associated_token_account(bob_pubkey)

In [37]:
import time
http_client.request_airdrop(payer_loaded_account.public_key, 1000000000)
# time.sleep(20)
# http_client.request_airdrop(alice_pubkey, 1000000000)
# time.sleep(20)
# http_client.request_airdrop(bob_pubkey, 1000000000)
# print(http_client.get_balance(alice_pubkey))
# print(http_client.get_minimum_balance_for_rent_exemption(7)["result"])

{'jsonrpc': '2.0',
 'result': 'nwTJE9WynKqjnhsapQrWk4LsR17FZ7XssET8hE2yzKAHcfKgMtngnGhcfq6H929FGZWpHhMmKu7d1rEd9NvYrkV',
 'id': 41}

In [38]:
# escrow meta-data account
deployed_program_key = glob.glob("../target/deploy/*.json")[0]
deployed_program_key_account = solana.keypair.Keypair(json.load(open(deployed_program_key))[:32])
program_id = deployed_program_key_account.public_key


tx = Transaction()
escrow_keypair = solana.keypair.Keypair.generate()
escrow_pubkey = escrow_keypair.public_key
data_size = 100
lamports = http_client.get_minimum_balance_for_rent_exemption(data_size)["result"]
transaction_instruction = create_account(
        params=CreateAccountParams(
            from_pubkey=payer_public_key, new_account_pubkey=escrow_pubkey,
            lamports=lamports, space=data_size, program_id=program_id
        )
    )

tx = tx.add(transaction_instruction)
transaction_results = http_client.send_transaction(tx, *[payer_loaded_account,escrow_keypair])
print(f"Program ID: {program_id}")

Program ID: G9x3V3ocLx8KKkWe3iEdwQRcWZKcqSu3A6z33vvGJqVU


In [39]:
# todo, creat seed more specific
x_seeds = [
    b"vault_x",
    bytes(alice_pubkey),
    bytes(bob_pubkey),
    bytes(X_mint_account_address.pubkey),
    bytes(Y_mint_account_address.pubkey),
]

y_seeds = [
    b"vault_y",
    bytes(alice_pubkey),
    bytes(bob_pubkey),
    bytes(X_mint_account_address.pubkey),
    bytes(Y_mint_account_address.pubkey),
]

escrow_seeds = [
    b"escrow",
    bytes(alice_pubkey),
    bytes(bob_pubkey),
    bytes(X_mint_account_address.pubkey),
    bytes(Y_mint_account_address.pubkey),
]


vaultx, xseed = PublicKey.find_program_address(seeds=x_seeds,program_id=program_id)
vaulty, yseed = PublicKey.find_program_address(seeds=y_seeds,program_id=program_id)
escrow_address, escrow_seed = PublicKey.find_program_address(seeds=escrow_seeds,program_id=program_id)

In [40]:
print(vaultx)
print(vaulty)
print(escrow_address)

G3Q6MrTHdUUjV3ceCQn3CzhkNBzVxKwNyHav5RK5Ramk
F1TVGusQ86yYSXTW1W4ZVcXi2e6aAQYJyyUUxEBgdSDb
26rpVUBZECXLfGPRuz9cfbKWG6rBLFXh5imcrXC8QzgA


In [48]:
from struct import *
data = pack('<BQQ', 0,12,15)

import struct
tx = Transaction()
tx_instruction = TransactionInstruction(
    keys=[AccountMeta(pubkey=escrow_address, is_signer=False, is_writable=True),
          AccountMeta(pubkey=X_mint_account_address.pubkey, is_signer=False, is_writable=False),
          AccountMeta(pubkey=Y_mint_account_address.pubkey, is_signer=False, is_writable=False),
          AccountMeta(pubkey=vaultx, is_signer=False, is_writable=True),
          AccountMeta(pubkey=vaulty, is_signer=False, is_writable=True),
          AccountMeta(pubkey=payer_public_key, is_signer=True, is_writable=False),
          AccountMeta(pubkey=alice_pubkey, is_signer=False, is_writable=False),
          AccountMeta(pubkey=bob_pubkey, is_signer=False, is_writable=False),
          AccountMeta(pubkey=TOKEN_PROGRAM_ID, is_signer=False, is_writable=False),
          AccountMeta(pubkey=SYSVAR_RENT_PUBKEY, is_signer=False, is_writable=False),
          AccountMeta(pubkey=SYS_PROGRAM_ID, is_signer=False, is_writable=False),
#           AccountMeta(pubkey=program_id, is_signer=False, is_writable=False),
         ],
    program_id=program_id,
    data=  data, #struct.pack("<LHB",1,1,1) # b58encode(bytes([9] * 7))#b'0001000100010001000100010001',
)

tx = tx.add(tx_instruction)


transaction_results = http_client.send_transaction(tx, *[payer_loaded_account])#, *[payer_loaded_account])

{'code': -32002, 'message': 'Transaction simulation failed: Error processing Instruction 0: custom program error: 0x0', 'data': {'accounts': None, 'err': {'InstructionError': [0, {'Custom': 0}]}, 'logs': ['Program G9x3V3ocLx8KKkWe3iEdwQRcWZKcqSu3A6z33vvGJqVU invoke [1]', 'Program log: Instruction: InitEscrow', 'Program log: Step 0', 'Program log: Step 1', 'Program log: Step 2', 'Program log: Step 3', 'Program log: Step 7', 'Program log: Step 9', 'Program log: Step 10', 'Program 11111111111111111111111111111111 invoke [2]', 'Create Account: account Address { address: G3Q6MrTHdUUjV3ceCQn3CzhkNBzVxKwNyHav5RK5Ramk, base: None } already in use', 'Program 11111111111111111111111111111111 failed: custom program error: 0x0', 'Program G9x3V3ocLx8KKkWe3iEdwQRcWZKcqSu3A6z33vvGJqVU consumed 200000 of 200000 compute units', 'Program G9x3V3ocLx8KKkWe3iEdwQRcWZKcqSu3A6z33vvGJqVU failed: custom program error: 0x0']}}


Exception: Failed to send transaction

In [44]:
escrow_address

26rpVUBZECXLfGPRuz9cfbKWG6rBLFXh5imcrXC8QzgA

In [61]:
a = http_client.get_account_info(escrow_address)['result']['value']['data'][0]

In [70]:
import base64
unpack('<QQB', base64.b64decode(a)[:17])

(12, 15, 1)

In [68]:
base64.b64decode(a)

b'\x0c\x00\x00\x00\x00\x00\x00\x00\x0f\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [67]:
a

'DAAAAAAAAAAPAAAAAAAAAAEAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA'